<a href="https://colab.research.google.com/github/Shirouky/lab6-Data-analysis/blob/main/LW6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Базовые функции

In [ ]:
import pandas as pd
import numpy as np
import time

In [ ]:
def import_xlsx():
  FILE_NAME = "DataFrame beads.xlsx"
  df = pd.read_excel(FILE_NAME)
  return df

df = import_xlsx()

In [ ]:
def timer(func):
  def inner(*args):
    start = time.time()
    result = func(*args)
    end = time.time()
    print(f"Время выполнения: {end - start:.4f} сек")

    return result
  return inner

# Медленные и быстрые функции

In [ ]:
@timer
def analyze_profit_slow(df, column="code_category"):
    profit = {}
    top_categories = df[column].value_counts().head(10).index
    profit = pd.DataFrame(index=top_categories)

    for category in top_categories:
        mask = df[column] == category
        category_data = df.loc[mask]

        profit.loc[category, "revenue"] = (category_data["price"] * category_data["quantity"]).sum()
        profit.loc[category, "avg_price"] = category_data["price"].mean()
        profit.loc[category, "quantity"] = category_data["quantity"].sum()
        profit.loc[category, "product_count"] = len(category_data)

    return profit

In [ ]:
@timer
def analyze_profit_fast(df, column="code_category"):
    df["revenue"] = df["price"] * df["quantity"]

    top_categories = df[column].value_counts().head(10).index
    profit = df[df[column].isin(top_categories)].groupby(column).agg({
        "revenue": "sum",
        "price": "mean",
        "quantity": "sum",
        "id_web": "count"
    }).rename(columns={"id_web": "product_count", "price": "avg_price"}).round(2)

    return profit

In [ ]:
@timer
def analyze_discount_slow(df):
  discount_effectiveness = 0
  for i in range(len(df)):
      if df.iloc[i]["sale"] < -5:
          discount_effectiveness += df.iloc[i]["price"] * df.iloc[i]["quantity"]

  return discount_effectiveness

In [ ]:
@timer
def analyze_discount_fast(df):
  discount_effectiveness = df[df["sale"] < -5]["revenue"].sum()

  return discount_effectiveness

# Вывод

In [ ]:
def print_profit(profit, title="ПРИБЫЛЬНОСТЬ ПО ТОП-10 КАТЕГОРИЯМ"):
    print(f"\n{title}")
    print('=' * 70)
    print(f"{"Категория":<12} {"Выручка, руб":<15} {"Ср. цена":<10} {"Кол-во":<8} {"Товаров":<8}")
    print('-' * 70)

    for category, metrics in profit.iterrows():
        revenue = f"{metrics["revenue"]:,.0f}"
        avg_price = f"{metrics["avg_price"]:.0f}"
        quantity = f"{metrics["quantity"]:,.0f}"
        product_count = f"{metrics["product_count"]:,.0f}"
        print(f"{category:<12} {revenue:<15} {avg_price:<10} {quantity:<8} {product_count:<8}")

In [ ]:
def print_summary(profit, discount):
    total_revenue = profit["revenue"].sum()
    discount_percentage = (discount / total_revenue) * 100

    best_revenue_category = profit.loc[profit["revenue"].idxmax()]

    print(f"Общая выручка топ-10 категорий: {total_revenue} руб")
    print(f"Доля продаж со скидкой: {discount_percentage:.1f}%")
    print(f"Категория с максимальной выручкой: {best_revenue_category.name} ({best_revenue_category['revenue']:,.0f} руб)")

In [ ]:
def compare():
    df = import_xlsx()

    print("МЕДЛЕННАЯ ВЕРСИЯ:")
    profit_slow = analyze_profit_slow(df)
    discount_slow = analyze_discount_slow(df)
    print_profit(profit_slow)
    print(f"\nВыручка от товаров со скидкой >5%: {discount_slow} руб")

    print("БЫСТРАЯ ВЕРСИЯ:")
    profit_fast = analyze_profit_fast(df)
    discount_fast = analyze_discount_fast(df)
    print_profit(profit_fast)
    print(f"\nВыручка от товаров со скидкой >5%: {discount_fast} руб")
    return profit_fast, discount_fast

In [ ]:
profit, discount = compare()
print_summary(profit, discount)

МЕДЛЕННАЯ ВЕРСИЯ:
Время выполнения: 0.0623 сек
Время выполнения: 0.2855 сек

ПРИБЫЛЬНОСТЬ ПО ТОП-10 КАТЕГОРИЯМ
Категория    Выручка, руб    Ср. цена   Кол-во   Товаров 
----------------------------------------------------------------------
24           409,610         167        2,392    149     
25           195,284         207        979      72      
5            208,763         211        985      65      
2            139,242         178        785      62      
20           158,962         176        918      54      
26           125,422         172        766      41      
63           164,665         194        829      39      
18           70,962          159        457      35      
59           78,644          179        439      33      
29           69,946          147        468      30      

Выручка от товаров со скидкой >5%: 3105793 руб
БЫСТРАЯ ВЕРСИЯ:
Время выполнения: 0.0209 сек
Время выполнения: 0.0014 сек

ПРИБЫЛЬНОСТЬ ПО ТОП-10 КАТЕГОРИЯМ
Категория    Выручка, р

In [ ]:
import pandas as pd
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt

def describe_dataset(df):
    """Описательная статистика данных"""
    print("=" * 50)
    print("1. ОПИСАНИЕ ДАННЫХ")
    print("=" * 50)

    print(f"Размер dataset: {df.shape}")
    print(f"Колонки: {df.columns.tolist()}")
    print("\nТипы данных:")
    print(df.dtypes)
    print("\nПропущенные значения:")
    print(df.isnull().sum())

    return df.shape

def calculate_metrics(df):
    """Расчет статистических метрик"""
    print("\n" + "=" * 50)
    print("2. СТАТИСТИЧЕСКИЕ МЕТРИКИ")
    print("=" * 50)

    metrics = {
        'mean_price': df['price'].mean(),
        'price_std': df['price'].std(),
        'price_cv': (df['price'].std() / df['price'].mean()) * 100,
        'discount_ratio': (df['sale'] < 0).sum() / len(df) * 100,
        'total_quantity': df['quantity'].sum()
    }

    print(f"Средняя цена: {metrics['mean_price']:.2f} руб")
    print(f"Коэффициент вариации цен: {metrics['price_cv']:.2f}%")
    print(f"Доля товаров со скидкой: {metrics['discount_ratio']:.1f}%")
    print(f"Общее количество: {metrics['total_quantity']:,} шт")

    return metrics

def check_normality(data1, data2):
    """Проверка нормальности распределения двух выборок"""
    _, p1 = stats.normaltest(data1)
    _, p2 = stats.normaltest(data2)
    return p1 > 0.05 and p2 > 0.05

def perform_statistical_test(data1, data2, is_normal):
    """Выполнение статистического теста"""
    if is_normal:
        stat, p_value = stats.ttest_ind(data1, data2, equal_var=False)
        test_name = "t-тест Стьюдента"
    else:
        stat, p_value = stats.mannwhitneyu(data1, data2)
        test_name = "U-тест Манна-Уитни"

    return test_name, stat, p_value

def hypothesis_discount_effect(df):
    """
    Гипотеза 1: Товары со скидкой >10% имеют большие объемы продаж
    Актуальность: Оптимизация стратегии скидок
    """
    print("\n" + "=" * 50)
    print("ГИПОТЕЗА 1: Влияние скидок на объемы продаж")
    print("=" * 50)

    high_discount = df[df['sale'] < -10]['quantity']
    low_discount = df[df['sale'] >= -10]['quantity']

    print(f"Группа со скидкой >10%: n={len(high_discount)}, mean={high_discount.mean():.2f}")
    print(f"Группа со скидкой ≤10%: n={len(low_discount)}, mean={low_discount.mean():.2f}")

    is_normal = check_normality(high_discount, low_discount)
    test_name, stat, p_value = perform_statistical_test(high_discount, low_discount, is_normal)

    print(f"Тест: {test_name}, p-value: {p_value:.4f}")
    return test_name, stat, p_value, high_discount.mean(), low_discount.mean()

def hypothesis_price_by_color(df):
    """
    Гипотеза 2: Товары премиальных цветовых категорий имеют более высокую цену
    Актуальность: Стратегия ценообразования по цветам
    """
    print("\n" + "=" * 50)
    print("ГИПОТЕЗА 2: Влияние цветовой категории на цену")
    print("=" * 50)

    # Определяем премиальные и стандартные цветовые категории
    color_price_means = df.groupby('color_category')['price'].mean()
    premium_threshold = color_price_means.quantile(0.75)  # верхние 25%

    premium_colors = color_price_means[color_price_means >= premium_threshold].index
    standard_colors = color_price_means[color_price_means < premium_threshold].index

    premium_prices = df[df['color_category'].isin(premium_colors)]['price']
    standard_prices = df[df['color_category'].isin(standard_colors)]['price']

    print(f"Премиальные цвета: n={len(premium_prices)}, mean={premium_prices.mean():.2f}")
    print(f"Стандартные цвета: n={len(standard_prices)}, mean={standard_prices.mean():.2f}")

    is_normal = check_normality(premium_prices, standard_prices)
    test_name, stat, p_value = perform_statistical_test(premium_prices, standard_prices, is_normal)

    print(f"Тест: {test_name}, p-value: {p_value:.4f}")
    return test_name, stat, p_value, premium_prices.mean(), standard_prices.mean()

def hypothesis_price_by_size(df):
    """
    Гипотеза 3: Меньшие размеры бисера имеют более высокую цену
    Актуальность: Понимание факторов ценообразования
    """
    print("\n" + "=" * 50)
    print("ГИПОТЕЗА 3: Влияние размера на цену")
    print("=" * 50)

    # Группируем по размерам (предполагаем, что меньшие размеры имеют большие номера)
    size_price_means = df.groupby('size')['price'].mean().sort_values(ascending=False)

    if len(size_price_means) >= 2:
        top_sizes = size_price_means.head(2).index
        bottom_sizes = size_price_means.tail(2).index

        expensive_sizes = df[df['size'].isin(top_sizes)]['price']
        cheap_sizes = df[df['size'].isin(bottom_sizes)]['price']

        print(f"Дорогие размеры {list(top_sizes)}: n={len(expensive_sizes)}, mean={expensive_sizes.mean():.2f}")
        print(f"Дешевые размеры {list(bottom_sizes)}: n={len(cheap_sizes)}, mean={cheap_sizes.mean():.2f}")

        is_normal = check_normality(expensive_sizes, cheap_sizes)
        test_name, stat, p_value = perform_statistical_test(expensive_sizes, cheap_sizes, is_normal)

        print(f"Тест: {test_name}, p-value: {p_value:.4f}")
        return test_name, stat, p_value, expensive_sizes.mean(), cheap_sizes.mean()
    else:
        print("Недостаточно размеров для анализа")
        return None, None, None, None, None

def interpret_results(results, alpha=0.05):
    """Интерпретация результатов всех гипотез"""
    print("\n" + "=" * 60)
    print("ОБЩИЕ ВЫВОДЫ ПО ИССЛЕДОВАНИЮ")
    print("=" * 60)

    conclusions = []

    for i, (test_name, stat, p_value, mean1, mean2) in enumerate(results, 1):
        if p_value is None:
            conclusions.append(f"Гипотеза {i}: Невозможно проверить (недостаточно данных)")
            continue

        if p_value < alpha:
            conclusion = f"Гипотеза {i}: ПОДТВЕРЖДЕНА (p={p_value:.4f})"
            if mean1 > mean2:
                conclusion += f" - разница: +{mean1-mean2:.2f}"
            else:
                conclusion += f" - разница: {mean1-mean2:.2f}"
        else:
            conclusion = f"Гипотеза {i}: НЕ ПОДТВЕРЖДЕНА (p={p_value:.4f})"

        conclusions.append(conclusion)

    for conclusion in conclusions:
        print(f"✓ {conclusion}")

    # Практические рекомендации
    print("\nПРАКТИЧЕСКИЕ РЕКОМЕНДАЦИИ:")
    for i, (test_name, stat, p_value, mean1, mean2) in enumerate(results, 1):
        if p_value and p_value < alpha:
            if i == 1 and mean1 > mean2:
                print("• Увеличить скидки >10% - они стимулируют продажи")
            elif i == 2 and mean1 > mean2:
                print("• Разработать премиальную линейку для дорогих цветов")
            elif i == 3 and mean1 > mean2:
                print("• Сфокусироваться на производстве мелких размеров - они более маржинальны")

def statistical_analysis(df):
    """
    Полное статистическое исследование данных о бисере
    """
    # 1. Описание данных
    describe_dataset(df)

    # 2. Расчет метрик
    metrics = calculate_metrics(df)

    # 3. Проверка гипотез
    results = []

    # Гипотеза 1: Влияние скидок
    result1 = hypothesis_discount_effect(df)
    results.append(result1)

    # Гипотеза 2: Влияние цвета
    result2 = hypothesis_price_by_color(df)
    results.append(result2)

    # Гипотеза 3: Влияние размера
    result3 = hypothesis_price_by_size(df)
    results.append(result3)

    # 4. Выводы и интерпретация
    interpret_results(results)

    return {
        'metrics': metrics,
        'hypotheses_results': results
    }

# ЗАПУСК ИССЛЕДОВАНИЯ
if __name__ == "__main__":
    # Загрузка данных
    df = import_xlsx()  # ваша функция загрузки данных

    # Запуск анализа
    results = statistical_analysis(df)

1. ОПИСАНИЕ ДАННЫХ
Размер dataset: (1083, 17)
Колонки: ['id_web', 'code', 'type', 'size', 'weight', 'color', 'sale', 'price', 'old_price', 'currency', 'quantity', 'image', 'url', 'color_category', 'type_category', 'code_category', 'color_code_category']

Типы данных:
id_web                  int64
code                   object
type                   object
size                   object
weight                  int64
color                  object
sale                    int64
price                   int64
old_price               int64
currency               object
quantity                int64
image                  object
url                    object
color_category          int64
type_category           int64
code_category           int64
color_code_category     int64
dtype: object

Пропущенные значения:
id_web                 0
code                   0
type                   0
size                   0
weight                 0
color                  0
sale                   0
price     